## Pulizia db

In [11]:
import glob
import pandas as pd
import os

# Imposta la directory principale dove cercare i file CSV
# Sostituisci 'C:/DEAM/' con il percorso reale della tua cartella principale
percorso_principale = r'C:\Users\Daniele\Desktop\DEAM_project'
os.chdir(percorso_principale)

# Trova tutti i file CSV nelle cartelle e sottocartelle
csv_files = glob.glob('**/*.csv', recursive=True)

# Crea un dizionario per memorizzare i DataFrame
dataframes = {}

# Carica ogni file CSV in un DataFrame
for file in csv_files:
    try:
        # Carica il file CSV in un DataFrame
        df = pd.read_csv(file)
        # Usa il percorso relativo completo come chiave per evitare conflitti
        dataframes[file] = df
        print(f"Caricato con successo: {file}")
    except Exception as e:
        print(f"Errore nel caricare {file}: {e}")

# Esempio: Mostra i file caricati e un'anteprima di uno specifico
print("\nFile CSV trovati e caricati:")
for file in dataframes.keys():
    print(f"- {file}")

# Esempio: Accedi a un DataFrame specifico e mostra le prime righe
esempio_file = r'C:\Users\Daniele\Desktop\DEAM_project\DEAM_Annotations\annotations averaged per song\dynamic (per second annotations)\arousal.csv'  # Sostituisci con un file esistente nel tuo caso
if esempio_file in dataframes:
    print(f"\nAnteprima di {esempio_file}:")
    print(dataframes[esempio_file].head())
else:
    print(f"\nIl file {esempio_file} non Ã¨ stato trovato.")

Caricato con successo: DEAM_Annotations\annotations averaged per song\dynamic (per second annotations)\arousal.csv
Caricato con successo: DEAM_Annotations\annotations averaged per song\dynamic (per second annotations)\valence.csv
Caricato con successo: DEAM_Annotations\annotations averaged per song\song_level\static_annotations_averaged_songs_1_2000.csv
Caricato con successo: DEAM_Annotations\annotations averaged per song\song_level\static_annotations_averaged_songs_2000_2058.csv
Caricato con successo: DEAM_Annotations\annotations per each rater\dynamic (per second annotations)\arousal\10.csv
Caricato con successo: DEAM_Annotations\annotations per each rater\dynamic (per second annotations)\arousal\1000.csv
Caricato con successo: DEAM_Annotations\annotations per each rater\dynamic (per second annotations)\arousal\1001.csv
Caricato con successo: DEAM_Annotations\annotations per each rater\dynamic (per second annotations)\arousal\1002.csv
Caricato con successo: DEAM_Annotations\annotatio

## struttura dataset

DEAM_PROJECT/
|--- DEAM_Annotations/
|    |--- annotations averaged per song/
|         |--- dynamic (per second annotations)/
|              |--- arousal.csv
|              |--- valence.csv
|         |--- song level/
|              |--- static_annotations_averaged_songs_1_2000.csv
|              |--- static_annotations_averaged_songs_2000_2058.csv
|    |--- annotations per each rater/
|         |--- dynamic (per second annotations)/
|              |--- arousal/
|                   |--- 2.csv
|                   |--- 3.csv
|                   |--- ...
|              |--- valence/
|                   |--- 2.csv
|                   |--- 3.csv
|                   |--- ...
|         |--- song level/
|              |--- static_annotations_songs_1_2000.csv/
|              |--- static_annotations_songs_2000_2058.csv/
|--- metadata/
|    |--- metadata_2013.csv
|    |--- metadata_2014.csv
|    |--- metadata_2015.csv
|--- features/
|    |--- 2.csv
|    |--- 3.csv
|    |--- ...
|--- DEAM_audio/
|    |--- 2.mp3
|    |--- 3.mp3
|    |--- ...

In [ ]:
# Esempio di "join" in Pandas su un campo comune, ad es. "song_id"

import pandas as pd

# Esempio: carico il CSV delle annotazioni di arousal (dinamiche, per secondo)
df_arousal = pd.read_csv(
    r"DEAM_Annotations\annotations averaged per song\dynamic (per second annotations)\arousal.csv"
)

# Carico il CSV delle annotazioni di valence (dinamiche, per secondo)
df_valence = pd.read_csv(
    r"DEAM_Annotations\annotations averaged per song\dynamic (per second annotations)\valence.csv"
)

# Esempio di "join" in Pandas su un campo comune, ad es. "song_id"
# (controlla come si chiama effettivamente la colonna ID nel tuo CSV)
df_join = pd.merge(df_arousal, df_valence, on="song_id", suffixes=("_ar", "_val"))

# Controlla i nomi delle colonne di df_join
print(df_join.columns)

# Esempio di "filtro" (equivalente a SELECT * FROM ... WHERE arousal > 0.5)
# Sostituisci 'sample_15000ms_ar' con il nome corretto della colonna
df_filtered = df_join.query("sample_15000ms_ar > 0.5")

# Esempio di "visualizzazione"
print(df_filtered.head())

# Esempio di "raggruppamento" (GROUP BY) e media
# Sostituisci 'sample_15000ms_ar' e 'sample_15000ms_val' con i nomi corretti delle colonne
df_grouped = df_join.groupby("song_id")[["sample_15000ms_ar", "sample_15000ms_val"]].mean()
print(df_grouped.head())

Index(['song_id', 'sample_15000ms_ar', 'sample_15500ms_ar',
       'sample_16000ms_ar', 'sample_16500ms_ar', 'sample_17000ms_ar',
       'sample_17500ms_ar', 'sample_18000ms_ar', 'sample_18500ms_ar',
       'sample_19000ms_ar',
       ...
       'sample_621500ms_val', 'sample_622000ms_val', 'sample_622500ms_val',
       'sample_623000ms_val', 'sample_623500ms_val', 'sample_624000ms_val',
       'sample_624500ms_val', 'sample_625000ms_val', 'sample_625500ms_val',
       'sample_626000ms_val'],
      dtype='object', length=2448)
    song_id  sample_15000ms_ar  sample_15500ms_ar  sample_16000ms_ar  \
16       25           0.502565           0.511875           0.509121   
21       39           0.555264           0.559063           0.563615   
61       84           0.582255           0.584212           0.587932   
62       85           0.570365           0.552331           0.568626   
89      115           0.656922           0.655746           0.633975   

    sample_16500ms_ar  sample_1700

In [ ]:
# Esempio di "query" SQL su DataFrame con pandasql

from pandasql import sqldf

# Seleziona solo le colonne necessarie
df_join_reduced = df_join[['song_id', 'sample_15000ms_ar', 'sample_15000ms_val']]

# Esempio di query SQL su DataFrame
q = """
SELECT a.song_id, a.sample_15000ms_ar as ar, a.sample_15000ms_val as val
FROM df_join_reduced a
WHERE a.sample_15000ms_ar > 0.5
ORDER BY a.sample_15000ms_ar DESC
LIMIT 5
"""

# Eseguo la query e ottengo un DataFrame
res = sqldf(q, globals())  # "globals()" serve per rendere i df_* visibili come tabelle
print(res)

   song_id     ar       val
0     1059  0.678 -0.122000
1     1667  0.675  0.485556
2     1398  0.673  0.454000
3     1254  0.672  0.091000
4     1992  0.659  0.084000
